In [63]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import numpy as np
from datetime import datetime
import time
import requests
from bs4 import BeautifulSoup

%matplotlib inline
df2008 = pd.read_csv('data/keiba_PS2008.csv')
df2009 = pd.read_csv('data/keiba_PS2009.csv')
df2010 = pd.read_csv('data/keiba_PS2010.csv')
df2011 = pd.read_csv('data/keiba_PS2011.csv')
df2012 = pd.read_csv('data/keiba_PS2012.csv')
df2013 = pd.read_csv('data/keiba_PS2013.csv')
df2014 = pd.read_csv('data/keiba_PS2014.csv')
df2015 = pd.read_csv('data/keiba_PS2015.csv')
df2016 = pd.read_csv('data/keiba_PS2016.csv')
df2017 = pd.read_csv('data/keiba_PS2017.csv')
df2018 = pd.read_csv('data/keiba_PS2018.csv')
df2019 = pd.read_csv('data/keiba_PS2019.csv')
df2020 = pd.read_csv('data/keiba_PS2020.csv')
df2021 = pd.read_csv('data/keiba_PS2021.csv')
df = pd.concat([df2008, df2009, df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020, df2021])
df.reset_index(drop=True, inplace=True)
#df = pd.concat([df2008, df2009, df2010, df2011])

In [64]:
#delete_index = df.index[(df["course"]=='障') | (df["rank"]=="中止") | (df["rank"]=='取消') | (df["rank"]=="失格") | (df['rank']=='除外') | pd.isna(df['rank'])]
#df.drop(delete_index, inplace=True)

In [65]:
df.isnull().sum()

Unnamed: 0       0
year             0
date             0
field            0
race             0
race_name      230
course           0
head_count     811
rank           811
horse_name     811
gender         811
age            811
trainerA      2290
trainerB      2255
weight        2041
c_weight      2255
jackie        2255
j_weight      2255
odds          3302
popu          3302
race_num         0
distance         0
time             0
cond             0
turn             0
goal_time      811
weather        811
wakuban       2255
umaban        2255
got_money     5990
dtype: int64

In [66]:
df[df.isnull().any(axis=1)].index

Int64Index([   104,    144,    198,    272,    286,    361,    390,    464,
               626,    627,
            ...
            674040, 674116, 674135, 674209, 674226, 674242, 674492, 674731,
            674842, 675085],
           dtype='int64', length=6263)

In [67]:
dic = {}

In [68]:
dic

{}

In [70]:
def numStr(num):
    if num >= 10:
        return str(num)
    else:
        return '0' + str(num)

Base = "http://race.sp.netkeiba.com/?pid=race_result&race_id="

key = 0

name = 'race_name'

for v, idx in enumerate(df[df.isnull().any(axis=1)].index):
    print(f'{v} / {len(df[df.isnull().any(axis=1)])}')
    d = df.iloc[idx]
    year = int(d['race_num'].astype(str)[:4])
    i = int(d['race_num'].astype(str)[4:6])
    j = int(d['race_num'].astype(str)[6:8])
    k = int(d['race_num'].astype(str)[8:10])
    l = int(d['race_num'].astype(str)[10:])
    url = Base + str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
    while 1:
        try:
            html = requests.get(url)
            break
        except:
            print('SLEEP!!!!')
            time.sleep(5)
            continue
    time.sleep(0.5)
    html.encoding = 'EUC-JP'
    # scraping
    soup = BeautifulSoup(html.text, 'html.parser')
    # ページがあるかの判定
    if soup.find_all('div', attrs={'class', 'Result_Guide'})!=[]:
        pass
    else:
        #共通部分を抜き出す
        CommonYear = year
        CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn'})[0].contents[1].contents[0]
        CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
        CommonRace = soup.find_all('div', attrs={'Race_Num'})[0].span.string
        CommonRname= soup.find_all('dt', attrs={'class', 'Race_Name'})[0].contents[0].strip()
        CommonCourse= soup.find_all('dd', attrs={'Race_Data'})[0].span.string
        CommonCourseDistance = soup.find_all('dd', attrs={'Race_Data'})[0].contents[2].contents[0]
        CommonTime = soup.find_all('dd', attrs={'Race_Data'})[0].contents[0]
        CommonWeather = soup.find_all('dd', attrs={'Race_Data'})[0].contents[4].contents[0]
        CommonHcount= soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[-1]
        CommonCond = soup.find_all('dd', attrs={'Race_Data'})[0].contents[-1].contents[0]
        CommonTurn = soup.find_all('dd', attrs={'Race_Data'})[0].contents[3][2]
        try:
            money_list = soup.find_all('div', attrs={'class', 'RaceList_Item01'})[1].contents[1].contents[2][4:-2].split('、')
        except:
            money_list = soup.find_all('div', attrs={'class', 'RaceList_Item01'})[1].contents[1].contents[1][4:-2].split('、')
        goal_time = soup.find_all('td', attrs=['class', 'Time'])
        rank = soup.find_all('div', attrs='Rank')
        horse_name = soup.find_all('dt', attrs=['class', 'Horse_Name'])
        detail_left = soup.find_all('span', attrs=['class', 'Detail_Left'])
        detail_right = soup.find_all('span', attrs=['class', 'Detail_Right'])
        odds = soup.find_all('td', attrs=['class', 'Odds'])
        waku = soup.find_all('td', attrs=['class', 'Num'])
        for m in range(len(soup.find_all('div', attrs='Rank'))):
            dst = pd.Series()
            try:
                dst['race_num'] = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
                dst['year'] = CommonYear
                dst['date'] = CommonDate
                dst['field']= CommonField #開催場所
                dst['race'] = CommonRace
                dst['race_name'] = CommonRname
                dst['course'] = CommonCourse
                dst['distance'] = CommonCourseDistance
                dst['time'] = CommonTime
                dst['cond'] = CommonCond
                dst['turn'] = CommonTurn
                if m == 0:
                    dst['goal_time'] = goal_time[0].contents[0]
                else:
                    try:
                        dst['goal_time'] = goal_time[m].contents[1].contents[1].contents[0]
                    except:
                        dst['goal_time'] = float('nan')
                dst['weather'] = CommonWeather
                dst['head_count'] = CommonHcount #頭数
                dst['rank'] = rank[m].contents[0]
                dst['horse_name'] = horse_name[m].a.string
                detailL = detail_left[m]
                dst['gender'] = list(detailL.contents[0].split()[0])[0]
                dst['age'] = list(detailL.contents[0].split()[0])[1]
                #dst['trainerA'] = detailL.span.string.split('･')[0]
                #dst['trainerB'] = detailL.span.string.split('･')[1]
                if len(detailL.contents)>=2:
                    try:
                        dst['weight'] = detailL.contents[3].contents[0][:-1].split('(')[0]
                    except:
                        dst['weight'] = float('nan')
                    try:
                        dst['c_weight'] = detailL.contents[3].contents[0][:-1].split('(')[1] #多分馬の体重変動
                    except:
                        dst['c_weight'] = float('nan')
                detailR = detail_right[m].contents
                try:
                    dst['jackie'] = detailR[0].strip().split(' ')[0]
                    dst['j_weight'] = detailR[0].strip().split(' ')[1] #多分jackieの体重変動
                    dst['trainerA'] = detailR[2].strip().split('･')[0]
                    dst['trainerB'] = detailR[2].strip().split('･')[1]
                except:
                    dst['jackie'] = detailR[1].contents[0]
                    dst['j_weight'] = float(detailR[2])
                    dst['trainerA'] = detailR[-1].strip().split('･')[0]
                    dst['trainerB'] = detailR[-1].strip().split('･')[1]
                Odds = odds[m].contents[1]
                if Odds.dt.string is not None:
                    dst['odds'] = Odds.dt.string.strip('倍')
                    dst['popu'] = Odds.dd.string.strip('人気') #何番人気か
                dst['wakuban'] = waku[2*m].contents[1].contents[0]
                dst['umaban'] = waku[2*m+1].contents[1].contents[0]
                try:
                    dst['got_money'] = money_list[int(dst['rank'])-1] if int(dst['rank']) <= len(money_list) else 0
                except:
                    dst['got_money'] = float('nan')
            except:
                import pdb; pdb.set_trace()
                print('passed')
            dst.name = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l) + numStr(m)
            dic[key] = dst.to_dict()
            key += 1

0 / 6263


/home/mil/masaoka/.pyenv/versions/anaconda3-2019.10/envs/keiba/lib/python3.7/site-packages/ipykernel_launcher.py:63: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


1 / 6263
2 / 6263
3 / 6263
4 / 6263
5 / 6263
6 / 6263
7 / 6263
8 / 6263
9 / 6263
10 / 6263
11 / 6263
12 / 6263
13 / 6263
14 / 6263
15 / 6263
16 / 6263
17 / 6263
18 / 6263
19 / 6263
20 / 6263
21 / 6263
22 / 6263
23 / 6263
24 / 6263
25 / 6263
26 / 6263
27 / 6263
28 / 6263
29 / 6263
30 / 6263
31 / 6263
32 / 6263
33 / 6263
34 / 6263
35 / 6263
36 / 6263
37 / 6263
38 / 6263
39 / 6263
40 / 6263
41 / 6263
42 / 6263
43 / 6263
44 / 6263
45 / 6263
46 / 6263
47 / 6263
48 / 6263
49 / 6263
50 / 6263
51 / 6263
52 / 6263
53 / 6263
54 / 6263
55 / 6263
56 / 6263
57 / 6263
58 / 6263
59 / 6263
60 / 6263
61 / 6263
62 / 6263
63 / 6263
64 / 6263
65 / 6263
66 / 6263
67 / 6263
68 / 6263
69 / 6263
70 / 6263
71 / 6263
72 / 6263
73 / 6263
74 / 6263
75 / 6263
76 / 6263
77 / 6263
78 / 6263
79 / 6263
80 / 6263
81 / 6263
82 / 6263
83 / 6263
84 / 6263
85 / 6263
86 / 6263
87 / 6263
88 / 6263
89 / 6263
90 / 6263
91 / 6263
92 / 6263
93 / 6263
94 / 6263
95 / 6263
96 / 6263
97 / 6263
98 / 6263
99 / 6263
100 / 6263
101 / 62

756 / 6263
757 / 6263
758 / 6263
759 / 6263
760 / 6263
761 / 6263
762 / 6263
763 / 6263
764 / 6263
765 / 6263
766 / 6263
767 / 6263
768 / 6263
769 / 6263
770 / 6263
771 / 6263
772 / 6263
773 / 6263
774 / 6263
775 / 6263
776 / 6263
777 / 6263
778 / 6263
779 / 6263
780 / 6263
781 / 6263
782 / 6263
783 / 6263
784 / 6263
785 / 6263
786 / 6263
787 / 6263
788 / 6263
789 / 6263
790 / 6263
791 / 6263
792 / 6263
793 / 6263
794 / 6263
795 / 6263
796 / 6263
797 / 6263
798 / 6263
799 / 6263
800 / 6263
801 / 6263
802 / 6263
803 / 6263
804 / 6263
805 / 6263
806 / 6263
807 / 6263
808 / 6263
809 / 6263
810 / 6263
811 / 6263
812 / 6263
813 / 6263
814 / 6263
815 / 6263
816 / 6263
817 / 6263
818 / 6263
819 / 6263
820 / 6263
821 / 6263
822 / 6263
823 / 6263
824 / 6263
825 / 6263
826 / 6263
827 / 6263
828 / 6263
829 / 6263
830 / 6263
831 / 6263
832 / 6263
833 / 6263
834 / 6263
835 / 6263
836 / 6263
837 / 6263
838 / 6263
839 / 6263
840 / 6263
841 / 6263
842 / 6263
843 / 6263
844 / 6263
845 / 6263
846 / 6263

1460 / 6263
1461 / 6263
1462 / 6263
1463 / 6263
1464 / 6263
1465 / 6263
1466 / 6263
1467 / 6263
1468 / 6263
1469 / 6263
1470 / 6263
1471 / 6263
1472 / 6263
1473 / 6263
1474 / 6263
1475 / 6263
1476 / 6263
1477 / 6263
1478 / 6263
1479 / 6263
1480 / 6263
1481 / 6263
1482 / 6263
1483 / 6263
1484 / 6263
1485 / 6263
1486 / 6263
1487 / 6263
1488 / 6263
1489 / 6263
1490 / 6263
1491 / 6263
1492 / 6263
1493 / 6263
1494 / 6263
1495 / 6263
1496 / 6263
1497 / 6263
1498 / 6263
1499 / 6263
1500 / 6263
1501 / 6263
1502 / 6263
1503 / 6263
1504 / 6263
1505 / 6263
1506 / 6263
1507 / 6263
1508 / 6263
1509 / 6263
1510 / 6263
1511 / 6263
1512 / 6263
1513 / 6263
1514 / 6263
1515 / 6263
1516 / 6263
1517 / 6263
1518 / 6263
1519 / 6263
1520 / 6263
1521 / 6263
1522 / 6263
1523 / 6263
1524 / 6263
1525 / 6263
1526 / 6263
1527 / 6263
1528 / 6263
1529 / 6263
1530 / 6263
1531 / 6263
1532 / 6263
1533 / 6263
1534 / 6263
1535 / 6263
1536 / 6263
1537 / 6263
1538 / 6263
1539 / 6263
1540 / 6263
1541 / 6263
1542 / 6263
1543

...

3509 / 6263
3510 / 6263
3511 / 6263
3512 / 6263
3513 / 6263
3514 / 6263
3515 / 6263
3516 / 6263
3517 / 6263
3518 / 6263
3519 / 6263
3520 / 6263
3521 / 6263
3522 / 6263
3523 / 6263
3524 / 6263
3525 / 6263
3526 / 6263
3527 / 6263
3528 / 6263
3529 / 6263
3530 / 6263
3531 / 6263
3532 / 6263
3533 / 6263
3534 / 6263
3535 / 6263
3536 / 6263
3537 / 6263
3538 / 6263
3539 / 6263
3540 / 6263
3541 / 6263
3542 / 6263
3543 / 6263
3544 / 6263
3545 / 6263
3546 / 6263
3547 / 6263
3548 / 6263
3549 / 6263
3550 / 6263
3551 / 6263
3552 / 6263
3553 / 6263
3554 / 6263
3555 / 6263
3556 / 6263
3557 / 6263
3558 / 6263
3559 / 6263
3560 / 6263
3561 / 6263
3562 / 6263
3563 / 6263
3564 / 6263
3565 / 6263
3566 / 6263
3567 / 6263
3568 / 6263
3569 / 6263
3570 / 6263
3571 / 6263
3572 / 6263
3573 / 6263
3574 / 6263
3575 / 6263
3576 / 6263
3577 / 6263
3578 / 6263
3579 / 6263
3580 / 6263
3581 / 6263
3582 / 6263
3583 / 6263
3584 / 6263
3585 / 6263
3586 / 6263
3587 / 6263
3588 / 6263
3589 / 6263
3590 / 6263
3591 / 6263
3592

4192 / 6263
4193 / 6263
4194 / 6263
4195 / 6263
4196 / 6263
4197 / 6263
4198 / 6263
4199 / 6263
4200 / 6263
4201 / 6263
4202 / 6263
4203 / 6263
4204 / 6263
4205 / 6263
4206 / 6263
4207 / 6263
4208 / 6263
4209 / 6263
4210 / 6263
4211 / 6263
4212 / 6263
4213 / 6263
4214 / 6263
4215 / 6263
4216 / 6263
4217 / 6263
4218 / 6263
4219 / 6263
4220 / 6263
4221 / 6263
4222 / 6263
4223 / 6263
4224 / 6263
4225 / 6263
4226 / 6263
4227 / 6263
4228 / 6263
4229 / 6263
4230 / 6263
4231 / 6263
4232 / 6263
4233 / 6263
4234 / 6263
4235 / 6263
4236 / 6263
4237 / 6263
4238 / 6263
4239 / 6263
4240 / 6263
4241 / 6263
4242 / 6263
4243 / 6263
4244 / 6263
4245 / 6263
4246 / 6263
4247 / 6263
4248 / 6263
4249 / 6263
4250 / 6263
4251 / 6263
4252 / 6263
4253 / 6263
4254 / 6263
4255 / 6263
4256 / 6263
4257 / 6263
4258 / 6263
4259 / 6263
4260 / 6263
4261 / 6263
4262 / 6263
4263 / 6263
4264 / 6263
4265 / 6263
4266 / 6263
4267 / 6263
4268 / 6263
4269 / 6263
4270 / 6263
4271 / 6263
4272 / 6263
4273 / 6263
4274 / 6263
4275

4875 / 6263
4876 / 6263
4877 / 6263
4878 / 6263
4879 / 6263
4880 / 6263
4881 / 6263
4882 / 6263
4883 / 6263
4884 / 6263
4885 / 6263
4886 / 6263
4887 / 6263
4888 / 6263
4889 / 6263
4890 / 6263
4891 / 6263
4892 / 6263
4893 / 6263
4894 / 6263
4895 / 6263
4896 / 6263
4897 / 6263
4898 / 6263
4899 / 6263
4900 / 6263
4901 / 6263
4902 / 6263
4903 / 6263
4904 / 6263
4905 / 6263
4906 / 6263
4907 / 6263
4908 / 6263
4909 / 6263
4910 / 6263
4911 / 6263
4912 / 6263
4913 / 6263
4914 / 6263
4915 / 6263
4916 / 6263
4917 / 6263
4918 / 6263
4919 / 6263
4920 / 6263
4921 / 6263
4922 / 6263
4923 / 6263
4924 / 6263
4925 / 6263
4926 / 6263
4927 / 6263
4928 / 6263
4929 / 6263
4930 / 6263
4931 / 6263
4932 / 6263
4933 / 6263
4934 / 6263
4935 / 6263
4936 / 6263
4937 / 6263
4938 / 6263
4939 / 6263
4940 / 6263
4941 / 6263
4942 / 6263
4943 / 6263
4944 / 6263
4945 / 6263
4946 / 6263
4947 / 6263
4948 / 6263
4949 / 6263
4950 / 6263
4951 / 6263
4952 / 6263
4953 / 6263
4954 / 6263
4955 / 6263
4956 / 6263
4957 / 6263
4958

5558 / 6263
5559 / 6263
5560 / 6263
5561 / 6263
5562 / 6263
5563 / 6263
5564 / 6263
5565 / 6263
5566 / 6263
5567 / 6263
5568 / 6263
5569 / 6263
5570 / 6263
5571 / 6263
5572 / 6263
5573 / 6263
5574 / 6263
5575 / 6263
5576 / 6263
5577 / 6263
5578 / 6263
5579 / 6263
5580 / 6263
5581 / 6263
5582 / 6263
5583 / 6263
5584 / 6263
5585 / 6263
5586 / 6263
5587 / 6263
5588 / 6263
5589 / 6263
5590 / 6263
5591 / 6263
5592 / 6263
5593 / 6263
5594 / 6263
5595 / 6263
5596 / 6263
5597 / 6263
5598 / 6263
5599 / 6263
5600 / 6263
5601 / 6263
5602 / 6263
5603 / 6263
5604 / 6263
5605 / 6263
5606 / 6263
5607 / 6263
5608 / 6263
5609 / 6263
5610 / 6263
5611 / 6263
5612 / 6263
5613 / 6263
5614 / 6263
5615 / 6263
5616 / 6263
5617 / 6263
5618 / 6263
5619 / 6263
5620 / 6263
5621 / 6263
5622 / 6263
5623 / 6263
5624 / 6263
5625 / 6263
5626 / 6263
5627 / 6263
5628 / 6263
5629 / 6263
5630 / 6263
5631 / 6263
5632 / 6263
5633 / 6263
5634 / 6263
5635 / 6263
5636 / 6263
5637 / 6263
5638 / 6263
5639 / 6263
5640 / 6263
5641

6241 / 6263
6242 / 6263
6243 / 6263
6244 / 6263
6245 / 6263
6246 / 6263
6247 / 6263
6248 / 6263
6249 / 6263
6250 / 6263
6251 / 6263
6252 / 6263
6253 / 6263
6254 / 6263
6255 / 6263
6256 / 6263
6257 / 6263
6258 / 6263
6259 / 6263
6260 / 6263
6261 / 6263
6262 / 6263


In [71]:
new = pd.DataFrame()
new = new.from_dict(dic, orient='index')
new.to_csv('data/temp.csv')

In [72]:
new.drop_duplicates(inplace=True)

In [73]:
new

,race_num,year,date,field,race,race_name,course,distance,time,cond,turn,goal_time,weather,head_count,rank,horse_name,gender,age,weight,c_weight,jackie,j_weight,trainerA,trainerB,odds,popu,wakuban,umaban,got_money
0,200801010108,2008,8/16(土),札幌,8R,3歳上500万下,ダ,1700m,14:50,稍,右,1:42.1,晴,13頭,1,ココシュニック,牝,3,502kg,+14,四位,52.0,栗東,松田国,3.2,1,6,9,740
1,200801010108,2008,8/16(土),札幌,8R,3歳上500万下,ダ,1700m,14:50,稍,右,1:46.4,晴,13頭,2,イノチノアカリ,牝,4,442kg,-2,武幸,55.0,栗東,武邦,4.8,2,5,7,300
2,200801010108,2008,8/16(土),札幌,8R,3歳上500万下,ダ,1700m,14:50,稍,右,1:47.0,晴,13頭,3,コパノタイマネ,牝,4,492kg,0,☆的場,54.0,栗東,崎山,5.4,4,6,8,190
3,200801010108,2008,8/16(土),札幌,8R,3歳上500万下,ダ,1700m,14:50,稍,右,1:47.1,晴,13頭,4,ブリズデロートンヌ,牝,3,446kg,+2,菊沢,52.0,美浦,鈴木康,14.0,6,4,5,110
4,200801010108,2008,8/16(土),札幌,8R,3歳上500万下,ダ,1700m,14:50,稍,右,1:47.4,晴,13頭,5,マルサンサイレンス,牝,3,452kg,-8,▲丸田,49.0,美浦,谷原,46.4,11,4,4,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90922,202110040808,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,1000m,13:55,稍,右,0:59.5,曇,13頭,9,アスクドンキバック,牡,3,460kg,+2,▲角田,52.0,栗東,梅田,71.0,12,6,9,0
90923,202110040808,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,1000m,13:55,稍,右,1:00.0,曇,13頭,10,フェブキラナ,牡,3,462kg,-2,幸,55.0,栗東,川村,11.8,5,5,6,0
90924,202110040808,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,1000m,13:55,稍,右,1:00.2,曇,13頭,11,スパークリングアイ,牡,3,488kg,-6,秋山真,55.0,栗東,大根田,63.6,11,3,3,0
90925,202110040808,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,1000m,13:55,稍,右,1:00.3,曇,13頭,12,ラストバリオン,牡,3,492kg,0,★永島,51.0,栗東,牧浦,53.5,9,4,4,0


In [74]:
new.isnull().sum()

race_num         0
year             0
date             0
field            0
race             0
race_name        0
course           0
distance         0
time             0
cond             0
turn             0
goal_time      811
weather          0
head_count       0
rank             0
horse_name       0
gender           0
age              0
weight        1339
c_weight      1553
jackie           0
j_weight         0
trainerA         0
trainerB         0
odds          2447
popu          2447
wakuban          0
umaban           0
got_money     5778
dtype: int64

In [75]:
race_nums = df[df.isnull().any(axis=1)]['race_num'].to_numpy()

In [76]:
drop = list(map(lambda x: x in race_nums, df['race_num']))
df[drop].index

Int64Index([    92,     93,     94,     95,     96,     97,     98,     99,
               100,    101,
            ...
            675076, 675077, 675078, 675079, 675080, 675081, 675082, 675083,
            675084, 675085],
           dtype='int64', length=74346)

In [77]:
delete_index = df[drop].index
df.drop(delete_index, inplace=True)
df = pd.concat([df, new])
df.reset_index(drop=True, inplace=True)
df.isnull().sum()

Unnamed: 0    74346
year              0
date              0
field             0
race              0
race_name         0
course            0
head_count        0
rank              0
horse_name        0
gender            0
age               0
trainerA          0
trainerB          0
weight         1339
c_weight       1553
jackie            0
j_weight          0
odds           2447
popu           2447
race_num          0
distance          0
time              0
cond              0
turn              0
goal_time       811
weather           0
wakuban           0
umaban            0
got_money      5778
dtype: int64

In [78]:
df

,Unnamed: 0,year,date,field,race,race_name,course,head_count,rank,horse_name,gender,age,trainerA,trainerB,weight,c_weight,jackie,j_weight,odds,popu,race_num,distance,time,cond,turn,goal_time,weather,wakuban,umaban,got_money
0,0.0,2008,8/16(土),札幌,1R,2歳未勝利,芝,14頭,1,ディアジーナ,牝,2.0,美浦,田村,468kg,0.0,四位,54.0,4.3,2.0,200801010101,1500m,10:40,良,右,1:29.3,曇,4.0,6.0,500.0
1,1.0,2008,8/16(土),札幌,1R,2歳未勝利,芝,14頭,2,アイディンパワー,牡,2.0,栗東,本田,460kg,0.0,藤田,54.0,5.2,4.0,200801010101,1500m,10:40,良,右,1:31.0,曇,8.0,14.0,200.0
2,2.0,2008,8/16(土),札幌,1R,2歳未勝利,芝,14頭,3,ウインリーズン,牡,2.0,美浦,加藤征,442kg,2.0,安藤勝,54.0,3.1,1.0,200801010101,1500m,10:40,良,右,1:31.4,曇,1.0,1.0,130.0
3,3.0,2008,8/16(土),札幌,1R,2歳未勝利,芝,14頭,4,ヴァンエボン,牡,2.0,栗東,山内,482kg,-4.0,小林徹,54.0,23.0,8.0,200801010101,1500m,10:40,良,右,1:31.5,曇,7.0,11.0,75.0
4,4.0,2008,8/16(土),札幌,1R,2歳未勝利,芝,14頭,5,ラヴェリータ,牝,2.0,栗東,松元,484kg,-20.0,長谷川,54.0,21.9,6.0,200801010101,1500m,10:40,良,右,1:31.6,曇,2.0,2.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675129,NaN,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,13頭,9,アスクドンキバック,牡,3,栗東,梅田,460kg,+2,▲角田,52.0,71.0,12,202110040808,1000m,13:55,稍,右,0:59.5,曇,6,9,0
675130,NaN,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,13頭,10,フェブキラナ,牡,3,栗東,川村,462kg,-2,幸,55.0,11.8,5,202110040808,1000m,13:55,稍,右,1:00.0,曇,5,6,0
675131,NaN,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,13頭,11,スパークリングアイ,牡,3,栗東,大根田,488kg,-6,秋山真,55.0,63.6,11,202110040808,1000m,13:55,稍,右,1:00.2,曇,3,3,0
675132,NaN,2021,9/5(日),小倉,8R,3歳以上1勝クラス,ダ,13頭,12,ラストバリオン,牡,3,栗東,牧浦,492kg,0,★永島,51.0,53.5,9,202110040808,1000m,13:55,稍,右,1:00.3,曇,4,4,0


In [79]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.to_csv('data/preprocessed_all.csv')

In [18]:
import requests
import time
from bs4 import BeautifulSoup


dst = ''
df_col = ['year', 'date', 'field', 'race', 'race_name'
          , 'course', 'head_count', 'rank', 'horse_name'
          , 'gender', 'age', 'trainerA', 'trainerB', 'weight', 'c_weight', 'jackie', 'j_weight'
          , 'odds','popu']

s = time.time()
key = 0

url = 'https://race.sp.netkeiba.com/?pid=race_result&race_id=200803030512'

while 1:
    try:
        html = requests.get(url)
        break
    except:
        print('SLEEP!!!!')
        time.sleep(5)
        continue
html.encoding = 'EUC-JP'
# scraping
soup = BeautifulSoup(html.text, 'html.parser')
# ページがあるかの判定
if soup.find_all('div', attrs={'class', 'Result_Guide'})!=[]:
    pass
else:
    #共通部分を抜き出す
    CommonYear = year
    CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn'})[0].contents[1].contents[0]
    CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
    CommonRace = soup.find_all('div', attrs={'Race_Num'})[0].span.string
    CommonRname= soup.find_all('dt', attrs={'class', 'Race_Name'})[0].contents[0].strip()
    CommonCourse= soup.find_all('dd', attrs={'Race_Data'})[0].span.string
    CommonCourseDistance = soup.find_all('dd', attrs={'Race_Data'})[0].contents[2].contents[0]
    CommonTime = soup.find_all('dd', attrs={'Race_Data'})[0].contents[0]
    CommonWeather = soup.find_all('dd', attrs={'Race_Data'})[0].contents[4].contents[0]
    CommonHcount= soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[-1]
    CommonCond = soup.find_all('dd', attrs={'Race_Data'})[0].contents[-1].contents[0]
    CommonTurn = soup.find_all('dd', attrs={'Race_Data'})[0].contents[3][2]
    try:
        money_list = soup.find_all('div', attrs={'class', 'RaceList_Item01'})[1].contents[1].contents[2][4:-2].split('、')
    except:
        money_list = soup.find_all('div', attrs={'class', 'RaceList_Item01'})[1].contents[1].contents[1][4:-2].split('、')
    goal_time = soup.find_all('td', attrs=['class', 'Time'])
    rank = soup.find_all('div', attrs='Rank')
    horse_name = soup.find_all('dt', attrs=['class', 'Horse_Name'])
    detail_left = soup.find_all('span', attrs=['class', 'Detail_Left'])
    detail_right = soup.find_all('span', attrs=['class', 'Detail_Right'])
    odds = soup.find_all('td', attrs=['class', 'Odds'])
    waku = soup.find_all('td', attrs=['class', 'Num'])
    for m in range(len(soup.find_all('div', attrs='Rank'))):
        if isinstance(rank[m].contents[0], str):
            continue
        dst = pd.Series(index=df_col)
        try:
            dst['race_num'] = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
            dst['year'] = CommonYear
            dst['date'] = CommonDate
            dst['field']= CommonField #開催場所
            dst['race'] = CommonRace
            dst['race_name'] = CommonRname
            dst['course'] = CommonCourse
            dst['distance'] = CommonCourseDistance
            dst['time'] = CommonTime
            dst['cond'] = CommonCond
            dst['turn'] = CommonTurn
            if m == 0:
                dst['goal_time'] = goal_time[0].contents[0]
            else:
                dst['goal_time'] = goal_time[m].contents[1].contents[1].contents[0]
            dst['weather'] = CommonWeather
            dst['head_count'] = CommonHcount #頭数
            dst['rank'] = rank[m].contents[0]
            dst['horse_name'] = horse_name[m].a.string
            detailL = detail_left[m]
            dst['gender'] = list(detailL.contents[0].split()[0])[0]
            dst['age'] = list(detailL.contents[0].split()[0])[1]
            #dst['trainerA'] = detailL.span.string.split('･')[0]
            #dst['trainerB'] = detailL.span.string.split('･')[1]
            if len(detailL.contents)>=2:
                dst['weight'] = detailL.contents[3].contents[0][:-1].split('(')[0]
                try:
                    dst['c_weight'] = detailL.contents[3].contents[0][:-1].split('(')[1] #多分馬の体重変動
                except:
                    dst['c_weight'] = float('nan')
            detailR = detail_right[m].contents
            try:
                dst['jackie'] = detailR[0].strip().split(' ')[0]
                dst['j_weight'] = detailR[0].strip().split(' ')[1] #多分jackieの体重変動
                dst['trainerA'] = detailR[2].strip().split('･')[0]
                dst['trainerB'] = detailR[2].strip().split('･')[1]
            except:
                dst['jackie'] = detailR[1].contents[0]
                dst['j_weight'] = float(detailR[2])
                dst['trainerA'] = detailR[-1].strip().split('･')[0]
                dst['trainerB'] = detailR[-1].strip().split('･')[1]
            Odds = odds[m].contents[1]
            if Odds.dt.string is not None:
                dst['odds'] = Odds.dt.string.strip('倍')
                dst['popu'] = Odds.dd.string.strip('人気') #何番人気か
            dst['wakuban'] = waku[2*m].contents[1].contents[0]
            dst['umaban'] = waku[2*m+1].contents[1].contents[0]
            dst['got_money'] = money_list[int(dst['rank'])-1] if int(dst['rank']) <= len(money_list) else 0
        except:
            print('passed')
            continue
        dst.name = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l) + numStr(m)
        #df = df.append(dst)
        dic[key] = dst.to_dict()
        key += 1

> /tmp/ipykernel_6142/842870015.py(40)<module>()
     38     #共通部分を抜き出す
     39     import pdb; pdb.set_trace()
---> 40     CommonYear = year
     41     CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn'})[0].contents[1].contents[0]
     42     CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()

ipdb> q


BdbQuit: 